# Calculs finance

## Traitement basique

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

Hello World


### Variable de calcul

In [53]:
def get_eur_to_usdc_rate():
    # API publique pour le taux EUR/USDC via CoinGecko
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "usd-coin", "vs_currencies": "eur"}  # ID pour USDC  # Contre EUR

    response = requests.get(url, params=params)
    data = response.json()

    # Récupère le taux EUR -> USDC
    rate = 1 / data["usd-coin"]["eur"]  # Inverser car API donne EUR -> USDC
    return rate

In [54]:
FLAT_TAXE = 0.33
INFLATION = 0.037
IR = 0.19
PS = 0.172
INTERET_LIVRET_A = 0.03
RATIO_EUR_USDC = get_eur_to_usdc_rate()

In [32]:
def my_print(row):
    return (
        row.drop(columns=["Type"])
        .style.format(
            {
                "Capital": "{:,.2f}€",
                "Apport": "{:,.2f}€",
                "Plus-Moins Value": "{:,.2f}€",
                "Performance": "{:.2%}",
            }
        )
        .hide()
    )


def print_revenu(row):
    return row.style.format(
        {
            "Montant": "{:,.2f}€",
        }
    ).hide()


def print_objectif(row):
    return (
        row.drop(columns=["Etat"])
        .style.format(
            {
                "Valeur": "{:,.2f}€",
                "Progression": "{:.2%}",
            }
        )
        .hide()
    )


def format_label(pct, all_vals):
    absolute = float(pct / 100 * sum(all_vals))
    return f"{pct:.2f}% {absolute:,.2f}€"

### Clean data

In [33]:
patrimoine = pd.read_csv("patrimoine.csv")
revenus_depenses = pd.read_csv("revenus-depenses.csv")
objectifs = pd.read_csv("objectif_long_terme.csv")

patrimoine["Capital"] = pd.to_numeric(patrimoine["Capital"])
patrimoine["Apport"] = pd.to_numeric(patrimoine["Apport"])

revenus_depenses["Montant"] = pd.to_numeric(revenus_depenses["Montant"])

objectifs["Valeur"] = pd.to_numeric(objectifs["Valeur"])
objectifs["Date"] = pd.to_datetime(objectifs["Date"], dayfirst=True)

### Traitement de la data

In [34]:
patrimoine = patrimoine.sort_values(by=["Capital"], ascending=False)

# Composition

In [ ]:
patrimoine_total = patrimoine["Capital"].sum()

plt.figure(figsize=(8, 6))
plt.pie(
    x=patrimoine["Capital"],
    labels=patrimoine["Actif"],
    autopct=lambda pct: format_label(pct, patrimoine["Capital"]),
    startangle=180,
)
plt.text(
    0,
    -1.5,
    f"Patrimoine total : {patrimoine_total:,.2f}€",
    ha="center",
    fontsize=12,
    color="black",
)
plt.title("Composition de mon patrimoine")
plt.show()

# Performance des investissements

In [36]:
def calcul_pmv(row):
    if row["Type"] != "livret":
        return row["Capital"] - row["Apport"]
    else:
        return 0


def calcul_perf(row):
    if row["Type"] != "livret":
        return row["Plus-Moins Value"] / row["Capital"]
    else:
        return INTERET_LIVRET_A

In [37]:
patrimoine["Plus-Moins Value"] = patrimoine.apply(calcul_pmv, axis=1)
patrimoine["Performance"] = patrimoine.apply(calcul_perf, axis=1)

In [ ]:
my_print(patrimoine)

5

In [39]:
patrimoine_melted = patrimoine.drop(
    columns=["Apport", "Type", "Capital", "Plus-Moins Value"]
).melt(id_vars="Actif", var_name="Type", value_name="Valeur")

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=patrimoine_melted, x="Actif", y="Valeur", hue="Type")

plt.title("Performance par Actif", fontsize=14)
plt.xlabel("Actif")
plt.ylabel("Valeur")
plt.legend(title="Type de mesure")

plt.show()

# Ceux-ci est un texte

In [41]:
print('salut')

salut


In [ ]:
nette_dinflation = patrimoine[["Actif"]].copy()
nette_dinflation["Type"] = patrimoine["Type"]
nette_dinflation["Plus-Moins Value"] = patrimoine.apply(ajout_inflation_pmv, axis=1)
nette_dinflation["Performance"] = patrimoine.apply(ajout_inflation_perf, axis=1)
my_print(nette_dinflation)

## Prise en compte de l'impôt

In [43]:
def calcul_impot_pmv(row):
    if row["Plus-Moins Value"] > 0:
        if row["Type"] == "boursier" or row["Type"] == "numerique":
            return row["Plus-Moins Value"] * (1 - FLAT_TAXE)
        elif row["Type"] == "foncier":
            return row["Plus-Moins Value"] * (1 - IR - PS)
        else:
            return row["Plus-Moins Value"]
    else:
        return row["Plus-Moins Value"]


def calcul_impot_perf(row):
    if row["Performance"] > 0:
        if row["Type"] == "boursier" or row["Type"] == "numerique":
            return row["Performance"] * (1 - FLAT_TAXE)
        elif row["Type"] == "foncier":
            return row["Performance"] * (1 - IR - PS)
        else:
            return row["Performance"]
    else:
        return row["Performance"]

In [ ]:
nette_dimpot = patrimoine[["Actif"]].copy()
nette_dimpot["Type"] = patrimoine["Type"]
nette_dimpot["Plus-Moins Value"] = nette_dinflation.apply(calcul_impot_pmv, axis=1)
nette_dimpot["Performance"] = nette_dinflation.apply(calcul_impot_perf, axis=1)
my_print(nette_dimpot)

# Revenus et dépenses

### Traitement de la data

In [45]:
revenus_depenses = revenus_depenses.sort_values(by=["Montant"], ascending=False)

## Répartition des dépenses

In [ ]:
depense = revenus_depenses.loc[revenus_depenses["Type"] == "depense"]
plt.figure(figsize=(8, 6))
plt.pie(
    x=depense["Montant"],
    labels=depense["Nom"],
    autopct=lambda pct: format_label(pct, depense["Montant"]),
    startangle=180,
)
plt.title("Répartition des dépenses")
plt.show()

# Indicateurs et ratios

## Calcul des indicateurs et ratios

In [57]:
def convert_eur_to_usdc(euros):
    return euros * RATIO_EUR_USDC

In [47]:
revenu_total = revenus_depenses.loc[
    revenus_depenses["Type"] == "revenu", "Montant"
].sum()
depense_total = revenus_depenses.loc[
    revenus_depenses["Type"] == "depense", "Montant"
].sum()
epargne = revenu_total - depense_total
taux_depargne = epargne / revenu_total

pourcentage_don = depense["Montant"].values[1] / revenu_total
ratio_liquidite = (
    patrimoine.loc[patrimoine["Type"] == "livret", "Capital"].sum() / depense_total
)

taux_depense_variable = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] == "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)
taux_depense_hebdomadaire = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] != "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)

In [58]:
trade_crypto_max = (
    patrimoine.loc[patrimoine["Type"] == "numerique", "Plus-Moins Value"].sum() / 2
)

trade_crypto_max_usdc = convert_eur_to_usdc(trade_crypto_max)


In [ ]:
print("Taux d'épargne: {:.2%}".format(taux_depargne))
print("Taux de dépense variable: {:.2%}".format(taux_depense_variable))
print("Taux de dépense hebdomadaire: {:.2%}".format(taux_depense_hebdomadaire))
print("Pourcentage de don: {:.2%}".format(pourcentage_don))
print("Ratio de liquidité: {:.2}".format(ratio_liquidite))
print("Trade USDC maximal acceptable: {:.2f}".format(trade_crypto_max_usdc))

# Objectifs

In [50]:
def calcul_progression(row):
    if (patrimoine_total / row["Valeur"]) > 1:
        return 1
    else:
        return patrimoine_total / row["Valeur"]

In [ ]:
objectifs["Progression"] = objectifs.apply(calcul_progression, axis=1)
print_objectif(objectifs)